In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from datetime import *
from dateutil.parser import parse

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

In [2]:
sc

## Contents

#### 0. Prerequisites
#### 1. Create the Schema
#### 2. Functions to parse and cleanse
#### 3. Create parsed & validated RDDs
#### 4. Save clean csv files into s3
#### 5. Create a dataframe and sanity check the whole process

## 0. Prerequsites

1. Setup the spark cluster
2. Upload this notebook
3. When you start pyspark, use the following command instead. This is required for loading the spark-csv framework used by data frames. This library is used in the last section for sanity checking.
>pyspark --packages com.databricks:spark-csv_2.10:1.3.0

## 1. Create the Schema (to be used in Dataframe)

Note the difference between yellow and green cabs: Green cab data has extra fields

1. Trip_type (indicating street hail or dispatched)
2. ehail_fee

**Yellow cab data header: ** 'VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount'

**Green cab data header: ** VendorID	lpep_pickup_datetime	Lpep_dropoff_datetime	Store_and_fwd_flag	RateCodeID	Pickup_longitude	Pickup_latitude	Dropoff_longitude	Dropoff_latitude	Passenger_count	Trip_distance	Fare_amount	Extra	MTA_tax	Tip_amount	Tolls_amount	Ehail_fee	improvement_surcharge	Total_amount	Payment_type	Trip_type 

#### 1. Yellow cab schema

In [3]:
#yCabFnameToGetHeader = "s3://testsetu/nyc/yellow_tripdata_2015-02.csv"
#yCabHeader = sc.textFile(yCabFnameToGetHeader).first()
yCabHeader = "cab_company,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount"

#schemaString = header
fields = [StructField(field_name, StringType(), True) for field_name in yCabHeader.split(',')]
print "# of columns: ", len(fields)

#cab_company: StringType (i.e. no change reqd)

#vendor_id: StringType (i.e. no change reqd)

#tpep_pickup_datetime
fields[2].dataType = TimestampType()

#tpep_dropoff_datetime
fields[3].dataType = TimestampType()

#passenger_count
fields[4].dataType = IntegerType()

#trip_distance
fields[5].dataType = FloatType()

#pickup_longitude
fields[6].dataType = FloatType()

#pickup_latitude
fields[7].dataType = FloatType()

#RateCodeID: StringType (i.e. no change reqd)
#store_and_fwd_flag: StringType (i.e. no change reqd)

#dropoff_longitude
fields[10].dataType = FloatType()

#dropoff_latitude
fields[11].dataType = FloatType()

#payment_type : StringType (i.e. no change reqd)

#fare_amount
fields[13].dataType = FloatType()

#extra
fields[14].dataType = FloatType()

#mta_tax
fields[15].dataType = FloatType()

#tip_amount
fields[16].dataType = FloatType()

#tolls_amount
fields[17].dataType = FloatType()

#improvement_surcharge
fields[18].dataType = FloatType()

#total_amount
fields[19].dataType = FloatType()

yCabSchema = StructType(fields)

# of columns:  20


#### 2. Green cab schema

In [4]:
gCabHeader = "cab_company,vendor_id,pickup_datetime,dropoff_datetime,store_and_fwd_flag,rate_code_id,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type"

#schemaString = header
fields = [StructField(field_name, StringType(), True) for field_name in gCabHeader.split(',')]
print "# of columns: ", len(fields)

#cab_company: StringType (i.e. no change reqd)

#vendor_id: StringType (i.e. no change reqd)

#pickup_datetime
fields[2].dataType = TimestampType()

#dropoff_datetime: 
fields[3].dataType = TimestampType()

#store_and_fwd_flag: StringType (i.e. no change reqd): 4

#rate_code_id: StringType (i.e. no change reqd): 5

#pickup_longitude
fields[6].dataType = FloatType()

#pickup_latitude
fields[7].dataType = FloatType()

#dropoff_longitude
fields[8].dataType = FloatType()

#dropoff_longitude
fields[9].dataType = FloatType()

#passenger_count
fields[10].dataType = IntegerType()

#trip_distance
fields[11].dataType = FloatType()

#fare_amount
fields[12].dataType = FloatType()

#extra
fields[13].dataType = FloatType()

#mta_tax
fields[14].dataType = FloatType()

#tip_amount
fields[15].dataType = FloatType()

#tolls_amount
fields[16].dataType = FloatType()

#ehail_fee
fields[17].dataType = FloatType()

#improvement_surcharge
fields[18].dataType = FloatType()

#total_amount
fields[19].dataType = FloatType()

#payment_type: StringType (i.e. no change reqd)

#trip_type: StringType (i.e. no change reqd)

gCabSchema = StructType(fields)

# of columns:  22


### 2. Functions to Parse & Cleanse the data

In [5]:
def parseDateWithErrorHandling(dateTimeStr):
    
    parsedDateTime = None
    
    try:
        if (dateTimeStr is None) or (dateTimeStr == ""):
            return None        
        parsedDateTime = parse(dateTimeStr)
    except:
        return None

    return parsedDateTime

def parseIntWithErrorHandling(intStr):
    parsedInt = None    
    try:
        if intStr is None:
            return None        
        parsedInt = int(intStr)
    except:
        return None

    return parsedInt

def parseFloatWithErrorHandling(floatStr):
    parsedFloat = None    
    try:
        if floatStr is None:
            return None        
        parsedFloat = float(floatStr)
    except:
        return None

    return parsedFloat

#### 2.1 Parse Yellow Cab Data

##### Issues with Yellow Cab data

The older records (2014 and earlier) seems to be MISSING the improvement_surcharge. **This has been fixed in the below code. **

In [7]:
#Mandatory elements: pickup date, dropoff date and the lat/long for pickup/drop off: if these are empty then return None
#other elements are can be empty
def parseYellowCabLine(line):
    
    try:
        elements = line.split(",")
    except:
        return None
    
    newRecordFormat = False
    
    if len(elements) == 19:
        newRecordFormat = True
    elif len(elements) == 18:
        newRecordFormat = False
    else:
        return None        
    
    parsedElements = []
    
    #Cab company: Yellow or Green
    parsedElements.append("Y")
        
    #VendorID: StringType (i.e. no change reqd)
    #If it is a header record, then we skip it (Starts with "VendorId")
    item = str(elements[0])
    if item.startswith('Vendor') is False:        
         parsedElements.append(elements[0])
    else:
        return None
        
    #tpep_pickup_datetime
    pickupDateTime = parseDateWithErrorHandling(elements[1])
    if pickupDateTime is None:
        return None    
    parsedElements.append(pickupDateTime)

    #tpep_dropoff_datetime
    dropoffDateTime = parseDateWithErrorHandling(elements[2])
    if dropoffDateTime is None:
        return None    
    parsedElements.append(dropoffDateTime)    
    
    #passenger_count
    item =  parseIntWithErrorHandling(elements[3])
    if (item is None):
        item = 0
    parsedElements.append(item)    

    #trip_distance
    item =  parseFloatWithErrorHandling(elements[4])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)    

    #pickup_longitude
    item =  parseFloatWithErrorHandling(elements[5])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)

    #pickup_latitude
    item =  parseFloatWithErrorHandling(elements[6])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)
        
    #RateCodeID: StringType (i.e. no change reqd)  
    parsedElements.append(elements[7]) 
    
    #store_and_fwd_flag: StringType (i.e. no change reqd)
    parsedElements.append(elements[8]) 

    #dropoff_longitude
    item =  parseFloatWithErrorHandling(elements[9])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)

    #dropoff_latitude
    item =  parseFloatWithErrorHandling(elements[10])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)

    #payment_type    
    parsedElements.append(elements[11])

    #fare_amount
    item =  parseFloatWithErrorHandling(elements[12])
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #extra
    item =  parseFloatWithErrorHandling(elements[13])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #mta_tax
    item =  parseFloatWithErrorHandling(elements[14])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #tip_amount
    item =  parseFloatWithErrorHandling(elements[15])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #tolls_amount
    item =  parseFloatWithErrorHandling(elements[16])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #improvement_surcharge
    if (newRecordFormat):
        item =  parseFloatWithErrorHandling(elements[17])
        if (item is None):
            item = 0.0
        parsedElements.append(item)

    #total_amount
    if (newRecordFormat):
        item =  parseFloatWithErrorHandling(elements[18])    
    else:
        item =  parseFloatWithErrorHandling(elements[17])
    if (item is None):
        item = 0.0
    parsedElements.append(item)
    
    return parsedElements

#### 2.2 Parse Green Cab Data

**Issues with Green Cab data**

1. Some old records seem to have **lat/long as zero**. These records will be **skipped** in the parse/cleanse belpw
2. The newer records (2015 onwards) has an extra empty column - this will be ignored
3. The pickup/dropoff date time seems to be inconsistent for many records: example: (2013-10-01 00:00:00, 2013-10-01 09:05:05).
    **Notice the high duration?**

**The issues (1) and (2) are addressed in the code below - issue (3) needs to be handled during analysis**

In [8]:
#Mandatory elements: pickup date, dropoff date and the lat/long for pickup/drop off: if these are empty then return None
#other elements are can be empty
def parseGreenCabLine(line):
    
    try:
        elements = line.split(",")
    except:
        return None
    
    if len(elements) < 21:
        return None
    
    parsedElements = []
    #Cab company: Yellow or Green
    parsedElements.append("G") 
            
    #VendorID: StringType (i.e. no change reqd)
    #If it is a header record, then we skip it (Starts with "VendorId")
    item = str(elements[0])
    if item.startswith('Vendor') is False:        
         parsedElements.append(elements[0])
    else:
        return None
        
    #tpep_pickup_datetime
    pickupDateTime = parseDateWithErrorHandling(elements[1])
    if pickupDateTime is None:
        return None    
    parsedElements.append(pickupDateTime)

    #tpep_dropoff_datetime
    dropoffDateTime = parseDateWithErrorHandling(elements[2])
    if dropoffDateTime is None:
        return None    
    parsedElements.append(dropoffDateTime)    
    
    #store_and_fwd_flag: StringType (i.e. no change reqd)
    parsedElements.append(elements[3])
    
    #RateCodeID: StringType (i.e. no change reqd)  
    parsedElements.append(elements[4])
    
     #pickup_longitude
    item =  parseFloatWithErrorHandling(elements[5])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)

    #pickup_latitude
    item =  parseFloatWithErrorHandling(elements[6])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)
    
    #dropoff_longitude
    item =  parseFloatWithErrorHandling(elements[7])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)

    #dropoff_latitude
    item =  parseFloatWithErrorHandling(elements[8])
    if (item is None) or (item == 0):
        return None
    parsedElements.append(item)
    
    #passenger_count
    item =  parseIntWithErrorHandling(elements[9])
    if (item is None):
        item = 0
    parsedElements.append(item)    

    #trip_distance
    item =  parseFloatWithErrorHandling(elements[10])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)    

    #fare_amount
    item =  parseFloatWithErrorHandling(elements[11])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #extra
    item =  parseFloatWithErrorHandling(elements[12])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #mta_tax
    item =  parseFloatWithErrorHandling(elements[13])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #tip_amount
    item =  parseFloatWithErrorHandling(elements[14])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #tolls_amount
    item =  parseFloatWithErrorHandling(elements[15])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)
    
    #ehail_fee
    item =  parseFloatWithErrorHandling(elements[16])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)
    
    #improvement_surcharge
    item =  parseFloatWithErrorHandling(elements[17])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)

    #total_amount
    item =  parseFloatWithErrorHandling(elements[18])    
    if (item is None):
        item = 0.0
    parsedElements.append(item)       

    #payment_type    
    parsedElements.append(elements[19])
    
    #trip_type    
    parsedElements.append(elements[20])
    
    return parsedElements

## 3. Create parsed/validated RDDs

### 3.1 Parse Yellow Cab

In [7]:
#Raw count: for info purpose only
#sc.textFile("s3://testsetu/nyc/yellow/*").count()

415400276

In [63]:
yCabRDD = sc.textFile("s3://testsetu/nyc/yellow/*")
yCabParsedRDD = yCabRDD.map(lambda line: parseYellowCabLine(line)).filter(lambda x: x is not None)
#count after parsing

#Number of clean records
%time print "Yellow cab: Number of parsed & clean records = ", yCabParsedRDD.count()

Yellow cab: Number of parsed & clean records =  415396639
CPU times: user 492 ms, sys: 160 ms, total: 652 ms
Wall time: 1h 43min 43s


In [83]:
print "Number of dirty Yellow Cab records = ", 415400276- 415396639

Number of dirty Yellow Cab records =  3637


In [62]:
print yCabParsedRDD.take(2)

[[u'2', datetime.datetime(2015, 1, 15, 19, 5, 39), datetime.datetime(2015, 1, 15, 19, 23, 42), 1, 1.59, -73.993896484375, 40.7501106262207, u'1', u'N', -73.97478485107422, 40.75061798095703, 1.0, 12.0, 1.0, 0.5, 3.25, 0.0, 0.3, 17.05], [u'1', datetime.datetime(2015, 1, 10, 20, 33, 38), datetime.datetime(2015, 1, 10, 20, 53, 28), 1, 3.3, -74.00164794921875, 40.7242431640625, u'1', u'N', -73.99441528320312, 40.75910949707031, 1.0, 14.5, 0.5, 0.5, 2.0, 0.0, 0.3, 17.8]]


In [ ]:
# Save the cleaned data
yCabParsedRDD.saveAsTextFile("s3://testsetu/nyc/final/yellow/")

### 3.2 Sanity tests on smaller data sets (Yellow Cab): To ensure we are not skipping good records

##### Test on a file from 2015

In [60]:
ySmallRDD = sc.textFile("s3://testsetu/nyc/yellow/yellow_tripdata_2015-01.csv")
print "Raw count for small dataset (without header): ", ySmallRDD.count() - 1

Raw count for small dataset (without header):  12748986


In [61]:
ySmallParsedRDD = ySmallRDD.map(lambda line: parseYellowCabLine(line)).filter(lambda x: x is not None)
print "Parsed/Cleaned count: ", ySmallParsedRDD.count()

Parsed/Cleaned count:  12748986


In [62]:
ySmallParsedRDD.take(2)

[['Y',
  u'2',
  datetime.datetime(2015, 1, 15, 19, 5, 39),
  datetime.datetime(2015, 1, 15, 19, 23, 42),
  1,
  1.59,
  -73.993896484375,
  40.7501106262207,
  u'1',
  u'N',
  -73.97478485107422,
  40.75061798095703,
  u'1',
  12.0,
  1.0,
  0.5,
  3.25,
  0.0,
  0.3,
  17.05],
 ['Y',
  u'1',
  datetime.datetime(2015, 1, 10, 20, 33, 38),
  datetime.datetime(2015, 1, 10, 20, 53, 28),
  1,
  3.3,
  -74.00164794921875,
  40.7242431640625,
  u'1',
  u'N',
  -73.99441528320312,
  40.75910949707031,
  u'1',
  14.5,
  0.5,
  0.5,
  2.0,
  0.0,
  0.3,
  17.8]]

##### Test on a file from 2013

In [51]:
ySmallRDD = sc.textFile("s3://testsetu/nyc/yellow/yellow_tripdata_2013-01.csv")
print "Raw count for small dataset (without header): ", ySmallRDD.count() - 1

Raw count for small dataset (without header):  14776616


In [58]:
ySmallParsedRDD = ySmallRDD.map(lambda line: parseYellowCabLine(line)).filter(lambda x: x is not None)
print "Parsed/Cleaned count: ", ySmallParsedRDD.count()

Parsed/Cleaned count:  14776529


In [59]:
ySmallParsedRDD.take(2)

[['Y',
  u'CMT',
  datetime.datetime(2013, 1, 1, 15, 11, 48),
  datetime.datetime(2013, 1, 1, 15, 18, 10),
  4,
  1.0,
  -73.978165,
  40.757977,
  u'1',
  u'N',
  -73.98984,
  40.751173,
  u'CSH',
  6.5,
  0.0,
  0.5,
  0.0,
  0.0,
  7.0],
 ['Y',
  u'CMT',
  datetime.datetime(2013, 1, 6, 0, 18, 35),
  datetime.datetime(2013, 1, 6, 0, 22, 54),
  1,
  1.5,
  -74.00668,
  40.731781,
  u'1',
  u'N',
  -73.994499,
  40.750659,
  u'CSH',
  6.0,
  0.5,
  0.5,
  0.0,
  0.0,
  7.0]]

### 3.3 Parse Green Cab

In [69]:
#Raw count: for info purpose only
print "Raw count of all green cab records:" , sc.textFile("s3://testsetu/nyc/green/*").count()

Raw count of all green cab records: 26943879


In [9]:
gCabRDD = sc.textFile("s3://testsetu/nyc/green/*")
gCabParsedRDD = gCabRDD.map(lambda line: parseGreenCabLine(line)).filter(lambda x: x is not None)
#count after parsing

#Number of clean records
%time print "Green cab: Number of parsed & clean records = ", gCabParsedRDD.count()

Green cab: Number of parsed & clean records =  26869879
CPU times: user 44 ms, sys: 20 ms, total: 64 ms
Wall time: 7min 41s


In [84]:
print "Number of dirty Green Cab records = ", 26943879 - 26869879

Number of dirty Green Cab records =  74000


In [94]:
#Sanity check the saved file
#gSavedFileRDD = sc.textFile("s3://testsetu/nyc/final/green/final-green.csv/part-00000")
#gParsedSavedFileRDD = gSavedFileRDD.map(lambda line: parseGreenCabLine(line)).filter(lambda x: x is not None)
#gParsedSavedFileRDD.count()

#gCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="DROPMALFORMED", header='false').load('s3://testsetu/nyc/yellow_tripdata_2015-02.csv', schema = yCabSchema)


### 3.4 Sanity tests on smaller data sets (Green Cab): To ensure we are not skipping good records

##### Test on a file from 2015

In [70]:
gSmallRDD = sc.textFile("s3://testsetu/nyc/green/green_tripdata_2015-01.csv")
print "Raw count for small dataset (without header): ", gSmallRDD.count() - 1

Raw count for small dataset (without header):  1508501


In [76]:
gSmallParsedRDD = gSmallRDD.map(lambda line: parseGreenCabLine(line)).filter(lambda x: x is not None)
print "Parsed/Cleaned count: ", gSmallParsedRDD.count()

Parsed/Cleaned count:  1504083


In [77]:
gSmallParsedRDD.take(2)

[['G',
  u'2',
  datetime.datetime(2015, 1, 1, 0, 34, 42),
  datetime.datetime(2015, 1, 1, 0, 38, 34),
  u'N',
  u'1',
  -73.92259216308594,
  40.7545280456543,
  -73.91363525390625,
  40.76552200317383,
  1,
  0.88,
  5.0,
  0.5,
  0.5,
  0.0,
  0.0,
  None,
  0.3,
  6.3,
  u'2',
  u'1'],
 ['G',
  u'2',
  datetime.datetime(2015, 1, 1, 0, 34, 46),
  datetime.datetime(2015, 1, 1, 0, 47, 23),
  u'N',
  u'1',
  -73.95275115966797,
  40.677711486816406,
  -73.98152923583984,
  40.65897750854492,
  1,
  3.08,
  12.0,
  0.5,
  0.5,
  0.0,
  0.0,
  None,
  0.3,
  13.3,
  u'2',
  u'1']]

##### Test on a file from 2013

In [79]:
gSmallRDD = sc.textFile("s3://testsetu/nyc/green/green_tripdata_2013-10.csv")
print "Raw count for small dataset (without header): ", gSmallRDD.count() - 1

Raw count for small dataset (without header):  170009


In [80]:
gSmallParsedRDD = gSmallRDD.map(lambda line: parseGreenCabLine(line)).filter(lambda x: x is not None)
print "Parsed/Cleaned count: ", gSmallParsedRDD.count()

Parsed/Cleaned count:  169018


In [81]:
gSmallParsedRDD.take(1)

[['G',
  u'2',
  datetime.datetime(2013, 10, 1, 0, 0, 54),
  datetime.datetime(2013, 10, 1, 0, 41, 52),
  u'N',
  u'1',
  -73.9388198852539,
  40.74968719482422,
  -74.0289535522461,
  40.63574981689453,
  1,
  12.76,
  42.0,
  0.5,
  0.5,
  0.0,
  0.0,
  None,
  43.0,
  2.0,
  u'',
  u'']]

## 4. Save cleaned files

In [10]:
def stringify(input):
    if input is None:
        return ""
    else:
        return str(input)
    
def rddToCsv(itemList):
    return ','.join(stringify(item) for item in itemList)    

In [17]:
#Tmp: to be deleted
#yCabRDD = sc.textFile("s3://testsetu/nyc/yellow/*")
#yCabParsedRDD = yCabRDD.map(lambda line: parseYellowCabLine(line)).filter(lambda x: x is not None)

In [18]:
yCleanedCsvRDD = yCabParsedRDD.map(lambda x: rddToCsv(x))
%time yCleanedCsvRDD.repartition(36).saveAsTextFile("s3://testsetu/nyc/final/yellow/consolidated")

CPU times: user 540 ms, sys: 228 ms, total: 768 ms
Wall time: 1h 58min 12s


In [11]:
gCleanedCsvRDD = gCabParsedRDD.map(lambda x: rddToCsv(x))
%time gCleanedCsvRDD.repartition(10).saveAsTextFile("s3://testsetu/nyc/final/green/consolidated")

CPU times: user 48 ms, sys: 16 ms, total: 64 ms
Wall time: 8min 35s


## 5. Create dataframe & sanity checking of the the whole process

In [16]:
sqlContext = SQLContext(sc)
gCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="DROPMALFORMED", header='false').load('s3://testsetu/nyc/final/green/consolidated/pa*', schema = gCabSchema)
%time print "Count of # records from data frame: ", gCabDF.count()
%time gCabDF.show(1)

Count of # records from data frame:  26869879
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 25.4 s
+-----------+---------+--------------------+--------------------+------------------+------------+----------------+---------------+-----------------+----------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+
|cab_company|vendor_id|     pickup_datetime|    dropoff_datetime|store_and_fwd_flag|rate_code_id|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|
+-----------+---------+--------------------+--------------------+------------------+------------+----------------+---------------+-----------------+----------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+--------

In [21]:
#yCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="DROPMALFORMED", header='false').load('s3://testsetu/nyc/final/yellow/consolidated/part*', schema = yCabSchema)
yCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="PERMISSIVE", header='false').load('s3://testsetu/nyc/final/yellow/consolidated/part*', schema = yCabSchema)
%time print "Count of # records from data frame: ", yCabDF.count()
#%time gCabDF.show(1)

Count of # records from data frame:  407403053
CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 5min 24s


In [24]:
#12748986-75640815
print "# dirty yellow cab records: ", 415400276 - 407403053
#415396639-407403053

# dirty yellow cab records:  7997223


### Appendix (Some testing code - for future reference):

In [15]:
#Create dataframe from RDD
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#yCabDF = sqlContext.createDataFrame(yCabParsedRDD, yCabSchema)
gCabDF = sqlContext.createDataFrame(gCabParsedRDD, gCabSchema)
gCabDF.show(2)

+-----------+---------+--------------------+--------------------+------------------+------------+----------------+---------------+-----------------+----------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+
|cab_company|vendor_id|     pickup_datetime|    dropoff_datetime|store_and_fwd_flag|rate_code_id|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|
+-----------+---------+--------------------+--------------------+------------------+------------+----------------+---------------+-----------------+----------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+
|          G|        2|2013-08-01 17:22:...|2013-08-01 17:

In [89]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
#from com.databricks.spark import csv
sqlContext = SQLContext(sc)
#yCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="DROPMALFORMED", header='true').load('./tmplocaldata/small/y*.csv', schema = yCabSchema)
#yCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="DROPMALFORMED", header='true').load('s3://testsetu/nyc/yellow_tripdata_2015-02.csv', schema = yCabSchema)
#yCabDF = sqlContext.read.format('com.databricks.spark.csv').options( mode="DROPMALFORMED", header='true').load('s3://testsetu/nyc/yellow*.csv', schema = yCabSchema)
yCabDF.select('pickup_datetime', 'pickup_longitude').take(2)

#.write.format('com.databricks.spark.csv').save('newcars.csv')

In [53]:
#Test on local file
yLocalRDD = sc.textFile("./tmplocaldata/small/y1.csv")
print "raw count: ", yLocalRDD.count()
yLocalParsedRDD = yLocalRDD.map(lambda line: parseYellowCabLine(line)).filter(lambda x: x is not None)
print "Parsed count: ", yLocalParsedRDD.count()

raw count:  100
Parsed count:  95


In [58]:
yCleanedCsvRDD = yLocalParsedRDD.map(lambda x: rddToCsv(x))
print "Count: ", yCleanedCsvRDD.count()
#yCleanedCsvRDD.take(2)
yCleanedCsvRDD.repartition(1).saveAsTextFile("./tmplocaldata/aa1.csv")

Count:  95


In [99]:
# code for future reference
gCabDF = sqlContext.createDataFrame(gCabParsedRDD, gCabSchema)
# Save the cleaned data
#gCabDF.show(5)
gCabDF.repartition(1).write \
    .format('com.databricks.spark.csv') \
    .save('s3://testsetu/nyc/final/green/consolidated')